In [1]:
# Import libs
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

In [2]:
# Load pretrained Alexnet model
alexnet = models.alexnet(pretrained = True)

In [3]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])]
)
img_to_tensor = transforms.ToTensor()

In [4]:
# Loading train data
train_image_names = []
y_train = []
with open('../Outex-TC-00010-c/000/train.txt', 'r') as f:
    next(f)
    for row in f:
        pom = row.split()
        train_image_names.append(pom[0])
        y_train.append(int(pom[1] ))
        
num_train = len(y_train)

# Loading test data
test_image_names = []
y_test = []
with open('../Outex-TC-00010-c/000/test.txt', 'r') as f:
    next(f)
    for row in f:
        pom = row.split()
        test_image_names.append(pom[0])
        y_test.append(int(pom[1] )) 
        
num_test = len(y_test)

In [5]:
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
# Defining an identity mapping to be used for the final output layer of the Alexnet
class Identity(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return x

In [7]:
# Changing the output of the Alexnet model to output a 4096 dimensional feature vector
alexnet.classifier[6] = Identity()

In [8]:
# Obtaining features for train images
all_train_image_features = torch.zeros((len(y_train), 4096))
for i in range(num_train):
    if i % 48 == 0:
        print(str(round(i/num_train*100))+'%', end = ' ')
    img = Image.open('../Outex-TC-00010-c/images/' + train_image_names[i])
    img_tensor = img_to_tensor(img).unsqueeze(0)
    features = alexnet(img_tensor)
    all_train_image_features[i, :] = alexnet(img_tensor)   
print('100%')

0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%


In [9]:
# Obtaining features for test images
all_test_image_features = torch.zeros((len(y_test), 4096))
for i in range(num_test):
    if i % 192 == 0:
        print(str(round(i/num_test*100))+'%', end = ' ')
    img = Image.open('../Outex-TC-00010-c/images/' + test_image_names[i])
    img_tensor = img_to_tensor(img).unsqueeze(0)
    features = alexnet(img_tensor)
    all_test_image_features[i, :] = alexnet(img_tensor)
print('100%')

0% 5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75% 80% 85% 90% 95% 100%


In [10]:
# Save train data to Train_data.npz
np.savez('Train_data.npz', x_train_all_features = all_train_image_features.detach().numpy(), y_train = y_train )

# Save test data to Test_data.npz
np.savez('Test_data.npz',  x_test_all_features = all_test_image_features.detach().numpy(), y_test = y_test )